In [0]:
%run "../General/NB_Configuration"

In [0]:
spark.sql(f"use catalog {catalog}")

In [0]:
# %sql
# TRUNCATE TABLE crm.financial_account_address;

In [0]:
basetable = spark.sql('''
SELECT fee.*
FROM silver.financial_account_fee fee
INNER JOIN crm.financial_account acct
    ON fee.FinancialAccountNumber = acct.FinancialAccountNumber
    AND acct.CurrentRecord = fee.CurrentRecord
''')
basetable.createOrReplaceTempView("FA_fee_vw")
basetable.display()

In [0]:
basetable = spark.sql('''
SELECT service.*
FROM silver.financial_account_service service
INNER JOIN crm.financial_account acct
    ON service.FinancialAccount = acct.FinancialAccountNumber
    and acct.currentrecord = service.CurrentRecord
''')
basetable.createOrReplaceTempView("FA_service_vw")
basetable.display()

In [0]:
basetable = spark.sql('''
SELECT * FROM silver.product prod
        -- WHERE prod.currentrecord = 'Yes'
''')
basetable.createOrReplaceTempView("product_vw")
basetable.display()

In [0]:
%sql
select productcode , currentrecord, count(*) from product_vw
group by 1,2
having count(*) > 1


In [0]:
basetable = spark.sql('''
select * from (
SELECT address.*
FROM silver.financial_account_address address
INNER JOIN crm.financial_account acct
    ON acct.FinancialAccountNumber = address.FinancialAccountNumber
    AND acct.currentrecord = address.currentrecord
)a
    ''')
basetable.createOrReplaceTempView("address_vw")
basetable.display()

In [0]:
basetable = spark.sql('''
SELECT bal.*, COUNT(*) OVER (PARTITION BY bal.FinancialAccountId, bal.BalanceAsOfDate, bal.type) as duplicate_count
FROM silver.financial_account_balance bal
INNER JOIN crm.financial_account acct
    ON acct.FinancialAccountNumber = bal.FinancialAccountId 
    AND bal.currentrecord = acct.currentrecord
''')
basetable.createOrReplaceTempView("fa_balance_vw")
basetable.display()

In [0]:
basetable = spark.sql('''
SELECT * FROM silver.branch
    ''')
basetable.createOrReplaceTempView("branch_vw")

In [0]:
basetable = spark.sql('''
SELECT * FROM silver.service_xref
    ''')
basetable.createOrReplaceTempView("service_xref_vw")

In [0]:
%sql
select 'fee' as tablename,count(*) as Failures from fa_fee_vw
where financialaccountnumber not in(select FinancialAccountNumber from silver.financial_account)
and currentrecord = 'Yes'
UNION ALL
select 'service', count(*) as Failures from fa_service_vw
where FinancialAccount not in(select FinancialAccountNumber from silver.financial_account)
and currentrecord = 'Yes'
UNION ALL
select 'silverproduct' as tablename, count(*) as Failures from product_vw 
where ProductCODE not in (select productcode from silver.financial_account)
Union all
select 'address' as tablename, count(*) as Failures from address_vw
where FinancialAccountNumber not in (select financialaccountnumber from silver.financial_account_address)
Union all
select 'balance' as tablename, count(*) as Failures from fa_balance_vw
where FinancialAccountID not in (select financialaccountnumber from crm.financial_account)
;

#Write tables 

In [0]:
basetable = spark.sql('''SELECT * FROM fa_fee_vw''')
basetable.write.mode('overwrite').saveAsTable('crm.financial_account_fee')

basetable = spark.sql('''SELECT * FROM fa_service_vw''')
basetable.write.option("mergeSchema", "true").mode('overwrite').saveAsTable('crm.financial_account_service')

basetable = spark.sql('''SELECT * FROM product_vw''')
basetable.write.option("mergeSchema", "true").mode('overwrite').saveAsTable('crm.product')

basetable = spark.sql('''SELECT * FROM address_vw''')
basetable.write.mode('overwrite').saveAsTable('crm.financial_account_address')

# spark.sql('''Truncate table crm.financial_account_balance''')
basetable = spark.sql('''SELECT * FROM fa_balance_vw''')
basetable.write.option("mergeSchema", "true").mode('overwrite').saveAsTable('crm.financial_account_balance')

basetable = spark.sql('''SELECT * FROM branch_vw''')
basetable.write.mode('overwrite').saveAsTable('crm.branch')

basetable = spark.sql('''SELECT * FROM service_xref_vw''')
basetable.write.mode('overwrite').saveAsTable('crm.service_xref')

In [0]:
%sql
select count(1) from crm.financial_account_address

In [0]:
%sql
Select Type, Sum(Balance)
from silver.financial_account_balance
where currentrecord = 'Yes'
group by Type